In [2]:
import tensorflow as tf
from tensorflow import keras
import keras_tuner as kt

In [3]:
(img_train, label_train), (img_test, label_test) = keras.datasets.fashion_mnist.load_data()

In [4]:
img_train = img_train.astype('float32') / 255.0
img_test = img_test.astype('float32') / 255.0

In [5]:
def model_builder(hp):
  model = keras.Sequential()
  model.add(keras.layers.Flatten(input_shape=(28, 28)))

  hp_units = hp.Int('units', min_value=32, max_value=512, step=32)
  model.add(keras.layers.Dense(units=hp_units, activation='relu'))
  model.add(keras.layers.Dense(10))

  hp_learning_rate = hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4])

  model.compile(optimizer=keras.optimizers.Adam(learning_rate=hp_learning_rate),
                loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                metrics=['accuracy'])

  return model

In [6]:
tuner = kt.Hyperband(model_builder,
                     objective='val_accuracy',
                     max_epochs=5,
                     factor=3,
                     directory='my_dir',
                     project_name='intro_to_kt')

INFO:tensorflow:Reloading Tuner from my_dir/intro_to_kt/tuner0.json


In [7]:
stop_early = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5)

In [8]:
tuner.search(img_train, label_train, epochs=5, validation_split=0.2, callbacks=[stop_early])
best_hps=tuner.get_best_hyperparameters(num_trials=1)[0]

print(f"""
The hyperparameter search is complete. The optimal number of units in the first densely-connected
layer is {best_hps.get('units')} and the optimal learning rate for the optimizer
is {best_hps.get('learning_rate')}.
""")

INFO:tensorflow:Oracle triggered exit

The hyperparameter search is complete. The optimal number of units in the first densely-connected
layer is 288 and the optimal learning rate for the optimizer
is 0.001.



In [9]:
model = tuner.hypermodel.build(best_hps)
history = model.fit(img_train, label_train, epochs=5, validation_split=0.2)

val_acc_per_epoch = history.history['val_accuracy']
best_epoch = val_acc_per_epoch.index(max(val_acc_per_epoch)) + 1
print('Best epoch: %d' % (best_epoch,))

2023-02-22 10:01:05.303009: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2023-02-22 10:01:05.303044: W tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:265] failed call to cuInit: UNKNOWN ERROR (303)
2023-02-22 10:01:05.303061: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (IGN-BLR-LP-0412): /proc/driver/nvidia/version does not exist
2023-02-22 10:01:05.303319: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


Epoch 1/5
1500/1500 [==============================] - 4s 2ms/step - loss: 0.4986 - accuracy: 0.8250 - val_loss: 0.4062 - val_accuracy: 0.8596
Epoch 2/5
1500/1500 [==============================] - 4s 2ms/step - loss: 0.3711 - accuracy: 0.8654 - val_loss: 0.3570 - val_accuracy: 0.8702
Epoch 3/5
1500/1500 [==============================] - 3s 2ms/step - loss: 0.3344 - accuracy: 0.8773 - val_loss: 0.3573 - val_accuracy: 0.8743
Epoch 4/5
1500/1500 [==============================] - 3s 2ms/step - loss: 0.3095 - accuracy: 0.8868 - val_loss: 0.3400 - val_accuracy: 0.8812
Epoch 5/5
1500/1500 [==============================] - 3s 2ms/step - loss: 0.2906 - accuracy: 0.8909 - val_loss: 0.3288 - val_accuracy: 0.8811
Best epoch: 4


In [10]:
hypermodel = tuner.hypermodel.build(best_hps)
hypermodel.fit(img_train, label_train, epochs=best_epoch, validation_split=0.2)

Epoch 1/4
1500/1500 [==============================] - 4s 2ms/step - loss: 0.5022 - accuracy: 0.8213 - val_loss: 0.4249 - val_accuracy: 0.8400
Epoch 2/4
1500/1500 [==============================] - 3s 2ms/step - loss: 0.3711 - accuracy: 0.8649 - val_loss: 0.3524 - val_accuracy: 0.8717
Epoch 3/4
1500/1500 [==============================] - 3s 2ms/step - loss: 0.3335 - accuracy: 0.8785 - val_loss: 0.3916 - val_accuracy: 0.8639
Epoch 4/4
1500/1500 [==============================] - 3s 2ms/step - loss: 0.3078 - accuracy: 0.8860 - val_loss: 0.3642 - val_accuracy: 0.8680


In [11]:
eval_result = hypermodel.evaluate(img_test, label_test)
print("[test loss, test accuracy]:", eval_result)

313/313 [==============================] - 0s 764us/step - loss: 0.3865 - accuracy: 0.8608
[test loss, test accuracy]: [0.3864968717098236, 0.86080002784729]
